In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

os.chdir('./Comp7103_Assignment1_data_mining_and_NLP')

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
!git clone https://github.com/google-research/bert.git
!pip install bert-serving-server # server
!pip install bert-serving-client # client, independent of 'bert-serving-server'
!pip install transformers

fatal: destination path 'bert' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=f6f1ad1e58abd652ce537a6e57ab0d8baba3f2af29e30bb7a37a8fbec626d446
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built GPUtil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.1 M

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras import optimizers

import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score

import random
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, RobertaConfig
from transformers import pipeline
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import torchsummary

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
df = pd.read_csv("amazon/Reviews.csv")
df = df.dropna()
print(len(df))
df = df.sample(n=int(len(df)*0.02))
print(len(df))

568411
11368


In [ ]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
451238,451239,B003QDRJXY,A36GZ21TM5H9ZE,Dan,0,0,5,1329436800,I love this coconut oil.,I love the taste of this coconut oil. Coconut...
329410,329411,B001P555WC,ABGMCK027S4L7,"Abundantly Blessed ""Mommy of 3""",2,2,5,1254528000,Wonderful Chocolate!,I was pleasently suprised at the amount of cho...
161474,161475,B001EO76WG,A301LTNV6629WX,Christine,1,1,5,1230076800,The best organic pasta,BioNature organic pasta is excellent. They co...
468582,468583,B001CE6RW8,A2RMMLCU94BDFJ,"1995shopper ""Choose to be happy""",1,1,5,1218067200,My cat's hair grew back!,So convenient having the food delivered to my ...
426056,426057,B000TQEWM2,A2P2TACM8IM7Q4,"Reinhard Loeffl ""rloeffl""",2,2,5,1264809600,The Best,I have tried all the K-Cup coffee's and I alwa...


In [ ]:
robert_df = df.loc[:,['Text', 'Summary', 'HelpfulnessNumerator','Score']]
robert_df['Score'] -= 1
train_df, test_df = train_test_split(robert_df, test_size=0.2, random_state=3074)
robert_df.head()

,Text,Summary,HelpfulnessNumerator,Score
451238,I love the taste of this coconut oil. Coconut...,I love this coconut oil.,0,4
329410,I was pleasently suprised at the amount of cho...,Wonderful Chocolate!,2,4
161474,BioNature organic pasta is excellent. They co...,The best organic pasta,1,4
468582,So convenient having the food delivered to my ...,My cat's hair grew back!,1,4
426056,I have tried all the K-Cup coffee's and I alwa...,The Best,2,4


baseline

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=5, output_hidden_states=True)
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 128)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
epochs = 10
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)        
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

In [ ]:
def evaluate(dataloader_val):
    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':     batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

f1_scores = []
training_loss = []
validation_loss = []    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        model = model.to(device)
        outputs = model(**inputs)
        # print(inputs['labels'])
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'./bert_checkpopint/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    f1_scores.append(val_f1)
    validation_loss.append(val_loss)
    training_loss.append(loss_train_avg)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2274 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.8647427255442987
Validation loss: 0.954338432561364
F1 Score (Weighted): 0.6735806407234287


Epoch 2:   0%|          | 0/2274 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.710491011382622
Validation loss: 0.8104698198449737
F1 Score (Weighted): 0.7507078143749538


...


Epoch 8
Training loss: 0.18459610970931176
Validation loss: 1.8200484720061654
F1 Score (Weighted): 0.7555478489617062


Epoch 9:   0%|          | 0/2274 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.1302096727689012
Validation loss: 1.8381933993958368
F1 Score (Weighted): 0.7585738905729638


Epoch 10:   0%|          | 0/2274 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.08777775919766008
Validation loss: 1.9176218835061034
F1 Score (Weighted): 0.7597445319593349
